<a href="https://colab.research.google.com/github/Naren123k/naren-511323106701/blob/main/naren_p3_nm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("stock_data.csv", parse_dates=["Date"])
df.sort_values("Date", inplace=True)

# Basic sanity check
df.reset_index(drop=True, inplace=True)

# -----------------------------
# Feature Engineering
# -----------------------------

# 1. Moving Averages
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()

# 2. Relative Strength Index (RSI)
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI_14'] = calculate_rsi(df['Close'])

# 3. Momentum
df['Momentum'] = df['Close'] - df['Close'].shift(5)

# 4. Log Returns
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

# 5. Lag Features (useful for time series models)
for lag in range(1, 6):
    df[f'lag_{lag}'] = df['Close'].shift(lag)

# Drop NaNs caused by rolling and shifting
df.dropna(inplace=True)

# -----------------------------
# Feature Scaling (optional here, essential for ML models)
# -----------------------------
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['Close', 'MA_10', 'MA_20', 'RSI_14', 'Momentum']])
scaled_df = pd.DataFrame(scaled_features, columns=['Close_Scaled', 'MA_10_Scaled', 'MA_20_Scaled', 'RSI_14_Scaled', 'Momentum_Scaled'])

# Combine scaled and original for modeling
df = pd.concat([df.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis=1)

# Final dataframe ready for modeling
print(df.head())
